In [ ]:
import numpy as np
from sklearn.cluster import KMeans,MiniBatchKMeans
from numpy.linalg import norm
import pickle
class MergedIVFPQ:
    def __init__(self, d, nlist, m, bits_per_subvector):
        """
        Initialize the IVFPQ index.
        :param d: Dimensionality of the input vectors
        :param nlist: Number of Voronoi cells (clusters) in IVF
        :param m: Number of sub-vectors for Product Quantization (PQ)
        :param bits_per_subvector: Number of bits for each subvector's quantization
        """
        self.d = d
        self.nlist = nlist
        self.m = m
        self.k = 2 ** bits_per_subvector  # Number of centroids per subvector
        self.subvector_dim = d // m

        assert d % m == 0, "Dimensionality must be divisible by the number of subvectors (m)."

        # Initialize KMeans for IVF and PQ
        self.kmeans = MiniBatchKMeans(n_clusters=self.nlist, init='k-means++', n_init=20,batch_size=100000, max_iter=500 )
        self.pq_codebooks = [MiniBatchKMeans(n_clusters=self.k,init='k-means++', n_init=20,batch_size=100000, max_iter=500) for _ in range(m)]

        self.inverted_lists = {i: [] for i in range(nlist)}  # Inverted file structure

    def train(self, data):
        """
        Train the IVF and PQ models on the given data.
        :param data: Input data array of shape (N, D)
        """
        self.kmeans.fit(data)
        self.centroids = self.kmeans.cluster_centers_
        
        # Form the codebook for each subvector
        for i in range(self.m):
            subvectors = data[:, i * self.subvector_dim: (i + 1) * self.subvector_dim]
            self.pq_codebooks[i].fit(subvectors)

    def encode(self, data):
        """
        Encode data into PQ codes.
        :param data: Input data array of shape (N, D)
        :return: PQ codes for the data
        """
        cluster_ids = self.kmeans.predict(data)  # Predict cluster IDs for all data
        pq_codes = np.zeros((data.shape[0], self.m), dtype=np.int32) 

        for idx, cluster_id in enumerate(cluster_ids):
            residual = data[idx] - self.centroids[cluster_id]
            for i in range(self.m):
                subvector = residual[i * self.subvector_dim: (i + 1) * self.subvector_dim].reshape(1, -1)
                centroid_vectors = self.pq_codebooks[i].cluster_centers_

                subvector_norm = norm(subvector)
                centroid_norms = norm(centroid_vectors, axis=1)

                if subvector_norm > 0:
                    similarities = np.dot(centroid_vectors, subvector.T) / (centroid_norms[:, np.newaxis] * subvector_norm)
                    similarities[np.isnan(similarities)] = -np.inf  # Handle NaN values
                    pq_codes[idx, i] = np.argmax(similarities)
                else:
                    pq_codes[idx, i] = 0  # Assign a default centroid if the subvector is zero
# pq_code stores the index for the codebook centers 
            self.inverted_lists[cluster_id].append((idx, pq_codes[idx]))

        return pq_codes

    def encode_single(self, vector):
        """
        Encode a single query vector.
        :param vector: Query vector of shape (D,)
        :return: (cluster_id, pq_code) for the query vector
        """
        cluster_id = self.kmeans.predict(vector.reshape(1, -1))[0]
        residual = vector - self.centroids[cluster_id]
        pq_code = np.zeros((self.m,), dtype=np.int32)

        for i in range(self.m):
            subvector = residual[i * self.subvector_dim: (i + 1) * self.subvector_dim].reshape(1, -1)
            centroid_vectors = self.pq_codebooks[i].cluster_centers_

            subvector_norm = norm(subvector)
            centroid_norms = norm(centroid_vectors, axis=1)

            if subvector_norm > 0:
                similarities = np.dot(centroid_vectors, subvector.T) / (centroid_norms[:, np.newaxis] * subvector_norm)
                similarities[np.isnan(similarities)] = -np.inf  # Handle NaN values
                pq_code[i] = np.argmax(similarities)
            else:
                pq_code[i] = 0  # Assign a default centroid if the subvector is zero

        return cluster_id, pq_code

    def search(self, query, top_k, nprobe=1):
        """
        Search for the top_k nearest neighbors to the query.
        :param query: Query vector of shape (1, D)
        :param top_k: Number of nearest neighbors to return
        :param nprobe: Number of clusters to probe
        :return: List of indices of the nearest neighbors
        """
        query = query.reshape(1, -1)
        
        # Calculate distances to each centroid and get the closest clusters
        cluster_distances = np.linalg.norm(self.centroids - query, axis=1)
        nearest_clusters = np.argsort(cluster_distances)[:nprobe]
        nearest_centroid = self.centroids[nearest_clusters[0]]
        residual = query - nearest_centroid  # Residual vector
        pq_similarities = []

        # Iterate over the nearest clusters
        for cluster_id in nearest_clusters:
            candidates = self.inverted_lists[cluster_id]
            
            for idx, pq_code in candidates:
                # Reconstruct the vector from the PQ code
                reconstructed_vector = self.reconstruct(pq_code, cluster_id) # center values for the vector
                
                # Calculate similarity (or distance) between query and reconstructed vector
                similarity = self.compute_similarity(residual, reconstructed_vector)
                
                # Append the result (index and similarity)
                pq_similarities.append((idx, similarity))
        
        # Sort by similarity in descending order
        pq_similarities.sort(key=lambda x: x[1], reverse=True)
        
        # Return the indices of the top_k nearest neighbors
        return [idx for idx, _ in pq_similarities[:top_k]]

    def reconstruct(self, pq_code, cluster_id):
        """
        Reconstruct the original vector from the PQ code.
        :param pq_code: The PQ code (list of centroid indices for each subvector)
        :param cluster_id: The cluster ID where the data point belongs (used to access centroids)
        :return: The reconstructed vector
        """
        reconstructed_vector = []
        
        # Reconstruct the vector by taking centroids corresponding to the PQ code
        for i in range(self.m):
            centroid_sub = self.pq_codebooks[i].cluster_centers_[pq_code[i]]
            reconstructed_vector.append(centroid_sub)
        
        return np.hstack(reconstructed_vector)  # Concatenate subvectors to form the full vector

    def compute_similarity(self, query, reconstructed_vector):
        """
        Compute the similarity (e.g., cosine similarity) between the query and the reconstructed vector.
        :param query: The query vector
        :param reconstructed_vector: The reconstructed vector from the PQ code
        :return: The computed similarity
        """
        dot_product = np.dot(query, reconstructed_vector)
        norm_query = norm(query)
        norm_reconstructed = norm(reconstructed_vector)
        
        if norm_query > 0 and norm_reconstructed > 0:
            similarity = dot_product / (norm_query * norm_reconstructed)
        else:
            similarity = -1  # If either vector has zero norm, assign a very low similarity
        
        return similarity

    def save_model(self, filepath='ivfpq_model.dat'):
        """
        Save the trained IVFPQ model to a .dat file.
        :param filepath: Path to the file where the model will be saved
        """
        with open(filepath, 'wb') as f:
            pickle.dump({
                'centroids': self.centroids,
                'pq_codebooks': [codebook.cluster_centers_ for codebook in self.pq_codebooks],
                'inverted_lists': self.inverted_lists
            }, f)

    def load_model(self, filepath='ivfpq_model.dat'):
        """
        Load the IVFPQ model from a .dat file.
        :param filepath: Path to the file where the model is saved
        """
        with open(filepath, 'rb') as f:
            data = pickle.load(f)
            self.centroids = data['centroids']

            # Reinitialize and set the PQ codebook centroids
            for i, codebook_centers in enumerate(data['pq_codebooks']):
                self.pq_codebooks[i].cluster_centers_ = codebook_centers

            self.inverted_lists = data['inverted_lists']


In [ ]:
from typing import Annotated
import numpy as np
import os
from b import MergedIVFPQ

DB_SEED_NUMBER = 42
ELEMENT_SIZE = np.dtype(np.float32).itemsize
DIMENSION = 70

class VecDB:
    def __init__(self, database_file_path="saved_db.dat", index_file_path="index.dat", new_db=True, db_size=None) -> None:
        self.db_path = database_file_path
        self.index_path = index_file_path
        # self.ivfpq = IVFPQ(nlist=150, m=4, nbits=8)
        self.ivfpq =  MergedIVFPQ(d=DIMENSION,nlist=5000, m=35, bits_per_subvector=8)
        # self.ivfpq =  MergedIVFPQ(d=DIMENSION,nlist=10000, m=70, bits_per_subvector=8) 10**4 0.0 0.09

        self._build_index(self.get_all_rows())
        if new_db:
            if db_size is None:
                raise ValueError("You need to provide the size of the database")
            # delete the old DB file if exists
            if os.path.exists(self.db_path):
                os.remove(self.db_path)
            self.generate_database(db_size)
    
    def generate_database(self, size: int) -> None:
        rng = np.random.default_rng(DB_SEED_NUMBER)
        vectors = rng.random((size, DIMENSION), dtype=np.float32)
        self._write_vectors_to_file(vectors)
        self._build_index(vectors)
    
    def _write_vectors_to_file(self, vectors: np.ndarray) -> None:
        mmap_vectors = np.memmap(self.db_path, dtype=np.float32, mode='w+', shape=vectors.shape)
        mmap_vectors[:] = vectors[:]
        mmap_vectors.flush()
    
    def _get_num_records(self) -> int:
        return os.path.getsize(self.db_path) // (DIMENSION * ELEMENT_SIZE)
    
    def insert_records(self, rows: Annotated[np.ndarray, (int, 70)]):
        num_old_records = self._get_num_records()
        num_new_records = len(rows)
        full_shape = (num_old_records + num_new_records, DIMENSION)
        mmap_vectors = np.memmap(self.db_path, dtype=np.float32, mode='r+', shape=full_shape)
        mmap_vectors[num_old_records:] = rows
        mmap_vectors.flush()
        self._build_index(mmap_vectors)  # Rebuild index after inserting new records
    
    def get_one_row(self, row_num: int) -> np.ndarray:
        try:
            offset = row_num * DIMENSION * ELEMENT_SIZE
            mmap_vector = np.memmap(self.db_path, dtype=np.float32, mode='r', shape=(1, DIMENSION), offset=offset)
            return np.array(mmap_vector[0])
        except Exception as e:
            return f"An error occurred: {e}"
    
    def get_all_rows(self) -> np.ndarray:
        num_records = self._get_num_records()
        vectors = np.memmap(self.db_path, dtype=np.float32, mode='r', shape=(num_records, DIMENSION))
        return np.array(vectors)
    def get_batch(self, start_idx: int, end_idx: int) -> np.ndarray:
        """
        Load a batch of vectors from the database using indices.

        :param start_idx: Starting index (inclusive)
        :param end_idx: Ending index (exclusive)
        :return: A NumPy array containing the batch of vectors
        """
        try:
            num_records = self._get_num_records()
            # Ensure indices are within bounds
            if start_idx < 0 or end_idx > num_records or start_idx >= end_idx:
                raise IndexError("Invalid start or end index for batch retrieval.")

            # Calculate the number of rows to fetch and their byte offset
            num_rows = end_idx - start_idx
            offset = start_idx * DIMENSION * ELEMENT_SIZE

            # Memory-map the batch into memory
            mmap_vectors = np.memmap(self.db_path, dtype=np.float32, mode='r', 
                                     shape=(num_rows, DIMENSION), offset=offset)

            # Convert to a standard NumPy array for easier handling
            return np.array(mmap_vectors)
        except Exception as e:
            return f"An error occurred: {e}"
    def retrieve_old(self, query: Annotated[np.ndarray, (1, DIMENSION)], top_k = 5):
        scores = []
        num_records = self._get_num_records()
        # here we assume that the row number is the ID of each vector
        for row_num in range(num_records):
            vector = self.get_one_row(row_num)
            score = self._cal_score(query, vector)
            scores.append((score, row_num))
        # here we assume that if two rows have the same score, return the lowest ID
        scores = sorted(scores, reverse=True)[:top_k]
        return [s[1] for s in scores]
    def retrieve(self, query: Annotated[np.ndarray, (1, DIMENSION)], top_k=5):
        print("Retrieving")
        # candidates = self.ivfpq.search(query, top_k)
        # return [self.get_all_rows().tolist().index(candidate.tolist()) for candidate in candidates]
        # query /= np.linalg.norm(query, axis=1, keepdims=True)

        return self.ivfpq.search(query,top_k, nprobe=100)
    
    def _cal_score(self, vec1, vec2):
        dot_product = np.dot(vec1, vec2)
        norm_vec1 = np.linalg.norm(vec1)
        norm_vec2 = np.linalg.norm(vec2)
        cosine_similarity = dot_product / (norm_vec1 * norm_vec2)
        return cosine_similarity
    
    def _build_index(self, vectors: np.ndarray) -> None:
        """
        Incrementally builds the IVFPQ index using batches of vectors.
        """
        # vectors = self.get_all_rows()
        # vectors /= np.linalg.norm(vectors, axis=1, keepdims=True)
        print( "Building index")
        self.ivfpq.train(vectors)
        print("fitting")
        self.ivfpq.encode(vectors)
        print("done")

        # num_records = self._get_num_records()
        # step = max(num_records // 1, 1)
        # for i in range(0, num_records, step):
        #     batch_vectors = self.get_batch(i, min(i + step, num_records))
        #     batch_vectors /= np.linalg.norm(batch_vectors, axis=1, keepdims=True)

        #     print("Training IVFPQ index on batch...")
        #     self.ivfpq.train_batch(batch_vectors)
        #     print("Encoding batch...")
        #     self.ivfpq.encode_batch(batch_vectors)
        #     print("Finished batch.")


In [ ]:
# This snippet of code is to show you a simple evaluate for VecDB class, but the full evaluation for project on the Notebook shared with you.
import numpy as np
from vec_db import VecDB
import time
from dataclasses import dataclass
from typing import List

@dataclass
class Result:
    run_time: float
    top_k: int
    db_ids: List[int]
    actual_ids: List[int]

def run_queries(db, np_rows, top_k, num_runs):
    results = []
    for _ in range(num_runs):
        query = np.random.random((1,70))
        
        tic = time.time()
        db_ids = db.retrieve(query, top_k)
        toc = time.time()
        run_time = toc - tic
        
        tic = time.time()
        actual_ids = np.argsort(np_rows.dot(query.T).T / (np.linalg.norm(np_rows, axis=1) * np.linalg.norm(query)), axis= 1).squeeze().tolist()[::-1]
        toc = time.time()
        np_run_time = toc - tic
        
        results.append(Result(run_time, top_k, db_ids, actual_ids))
    return results
def run_queries_old(db, np_rows, top_k, num_runs):
    results = []
    for _ in range(num_runs):
        query = np.random.random((1,70))
        
        tic = time.time()
        db_ids = db.retrieve_old(query, top_k)
        toc = time.time()
        run_time = toc - tic
        
        tic = time.time()
        actual_ids = np.argsort(np_rows.dot(query.T).T / (np.linalg.norm(np_rows, axis=1) * np.linalg.norm(query)), axis= 1).squeeze().tolist()[::-1]
        toc = time.time()
        np_run_time = toc - tic
        
        results.append(Result(run_time, top_k, db_ids, actual_ids))
    return results

def eval(results: List[Result]):
    # scores are negative. So getting 0 is the best score.
    scores = []
    run_time = []
    for res in results:
        run_time.append(res.run_time)
        # case for retrieving number not equal to top_k, score will be the lowest
        if len(set(res.db_ids)) != res.top_k or len(res.db_ids) != res.top_k:
            scores.append( -1 * len(res.actual_ids) * res.top_k)
            continue
        score = 0
        for id in res.db_ids:
            try:
                ind = res.actual_ids.index(id)
                if ind > res.top_k * 3:
                    score -= ind
            except:
                score -= len(res.actual_ids)
        scores.append(score)

    return sum(scores) / len(scores), sum(run_time) / len(run_time)


if __name__ == "__main__":
    db = VecDB(db_size= 10**4)

    all_db = db.get_all_rows()

    res1 = run_queries(db, all_db, 5, 10)
    # res = run_queries_old(db, all_db, 5, 10)

    print(eval(res1))
    # print(eval(res))